## Learning bi-partite motifs based on a thermodynamic approach
### Implements the dynamic programming and the gradient descent

In [1]:
import os
import numpy as np
from matplotlib import pyplot as plt
import itertools
import pandas as pd
from scipy.optimize import fmin_l_bfgs_b
from scipy.optimize import check_grad
from scipy.special import logsumexp
from Bio import SeqIO
import random

%load_ext cython

#Load Robert Kern's line profiler
%load_ext line_profiler
import line_profiler

In [2]:
#Set compiler directives (cf. http://docs.cython.org/src/reference/compilation.html)
from Cython.Compiler.Options import get_directive_defaults
directive_defaults = get_directive_defaults()
directive_defaults['linetrace'] = True
directive_defaults['binding'] = True

### Import fasta files

In [3]:
def parse_fasta(file_name):
    input_seq_iterator = SeqIO.parse(file_name, "fasta")
    return [str(record.seq) for record in input_seq_iterator]

In [60]:
set_size = 100
bg = parse_fasta('HNRNPA0_1_TGTCGA40NCCGA_AAG_1.fasta.tmp')
plus = parse_fasta('HNRNPA0_4_TGTCGA40NCCGA_AAG_4.fasta.tmp')

bg = random.sample(bg, set_size)
plus = random.sample(plus, set_size)

In [61]:
bg   = [seq.replace('N', random.sample(['A','T','C','G'],1)[0]) for seq in bg]
plus = [seq.replace('N', random.sample(['A','T','C','G'],1)[0]) for seq in plus]

### cython

In [24]:
%%cython -f -I . --compile-args=-DCYTHON_TRACE=1 


cimport cython
import numpy as np
import itertools
from libc.math cimport exp,pow


cdef int l = 3 #l_A=l_B=3 nucleotides
cdef int l_p = 3 #persistence length is 3 nucleotides 
cdef double cpi = np.pi

cpdef generate_kmer_inx():
    cdef dict vals = {'A':0,'C':1,'G':2,'T':3}
    cdef dict kmer_inx = {}
    
    for p in list(itertools.product(vals.keys(), repeat=l)):
        inx = 0
        for j,base in enumerate(p):
            inx += (4**j)*vals[base] 
        kmer_inx[''.join(p)] = inx
    return kmer_inx

kmer_inx = generate_kmer_inx()
inx_kmer = {y:x for x,y in kmer_inx.items()}

cpdef seq2int_cy(str sequence):
    cdef int L = len(sequence)
    kmer_array = np.zeros(L, dtype=int)
    
    cdef i
    for i in range(l-1,L):
        kmer = sequence[i-l+1:i+1]
        kmer_array[i] = kmer_inx[kmer]
    return kmer_array        


cpdef void assign_za_cy(long[:] x, int i, double[:] za, double[:] zb, double[:] Ea, double[:] Eb, double cab, double D):
    if i == l-1:
        za[i] = cab * exp(-Ea[x[i]])
        return
    za[i] = zb[i-l] * cab * exp(-Ea[x[i]])
    
cpdef void assign_zb_cy(long[:] x, int i, double[:] za, double[:] zb, double[:] Ea, double[:] Eb, double cab, double D):
    cdef double z = zb[i-1]
    cdef int j
    
    if i == l-1:
        z += cab*exp(-Eb[x[i]])  
    else:
        for j in range(0,i-l+1):
            z += za[j]*cb_cy(i-j-l, cab, D)*exp(-Eb[x[i]])
            z += zb[j]*cab*np.exp(-Eb[x[i]])      
    zb[i] = z 


@cython.cdivision(True)
cpdef double cb_cy(int d, double cab, double D):
    
    if d < 0:
        return 0
    cdef double sig = 1.0 / (3.0*(d+1.0)*l_p)
    cdef double gaussian = exp(-pow(D,2.0) / (2.0 * pow(sig, 2.0))) / pow(2.0*cpi*pow(sig,2.0),3.0/2.0)
    return (cab + gaussian)


cpdef void assign_za_E_derivatives_cy(long[:] x, int i, int inx, double[:] za, double[:] zb,
                                 double[:,:] za_Ea_derivatives, double[:,:] zb_Ea_derivatives, double[:,:] za_Eb_derivatives, double[:,:] zb_Eb_derivatives,
                                 double[:] Ea, double[:] Eb, double cab, double D):
    identical = (inx == x[i])
    
    if i == l-1:
        za_Ea_derivatives[inx,i] = -identical*cab*exp(-Ea[x[i]])
        za_Eb_derivatives[inx,i] = 0
        return
    
    za_Ea_derivatives[inx,i] = cab*zb_Ea_derivatives[inx,i-l]*exp(-Ea[x[i]]) - cab*zb[i-l]*identical*exp(-Ea[x[i]])
    za_Eb_derivatives[inx,i] = cab*zb_Eb_derivatives[inx,i-l]*exp(-Ea[x[i]])

    

@cython.boundscheck(False) # Deactivate bounds checking
@cython.wraparound(False)    
def assign_zb_E_derivatives_cy(long[:] x, int i, int inx, double[:] za, double[:] zb,
                                 double[:,:] za_Ea_derivatives, double[:,:] zb_Ea_derivatives, double[:,:] za_Eb_derivatives, double[:,:] zb_Eb_derivatives,
                                 double[:] Ea, double[:] Eb, double cab, double D):
    cdef int identical = (inx == x[i])
    
    cdef double der_b = zb_Eb_derivatives[inx,i-1]
    cdef double der_a = zb_Ea_derivatives[inx,i-1]
    cdef int j
    
    if i == l-1:
        der_b += -cab*identical*exp(-Eb[x[i]])
        der_a += 0
        
    else:
        for j in range(0,i-l+1):
            der_b += cb_c(i-j-l, cab, D, l_p) * ((za_Eb_derivatives[inx,j]*exp(-Eb[x[i]]) - za[j]*exp(-Eb[x[i]])*identical))
            der_b += cab * (zb_Eb_derivatives[inx,j]*exp(-Eb[x[i]]) - zb[j]*exp(-Eb[x[i]])*identical)
            
            der_a += cb_c(i-j-l, cab, D, l_p) * za_Ea_derivatives[inx,j]*exp(-Eb[x[i]])
            der_a += cab * zb_Ea_derivatives[inx,j]*exp(-Eb[x[i]]) 

    
    zb_Ea_derivatives[inx,i] = der_a
    zb_Eb_derivatives[inx,i] = der_b

cdef extern from "assign_zb_E_derivatives.c":
    pass
    
cdef extern from "assign_zb_E_derivatives.h":
    cdef void assign_zb_E_derivatives_c(long* x, int i, int inx, double* za, double* zb, int L, int l, double l_p,
                                 double* za_Ea_derivatives, double* zb_Ea_derivatives, double* za_Eb_derivatives, double* zb_Eb_derivatives,
                                 double* Ea, double* Eb, double cab, double D)
    cdef double cb_c(int, double, double, double)
    
    
cpdef double cb_D_derivative_cy(int d, double D):
    if d < 0:
        return 0
    cdef double sig = 1 / (3*(d+1)*l_p)
    cdef double der = -(D * exp((-np.power(D,2))/(2*np.power(sig,2)))) / (np.power(2*cpi, 3/2)*np.power(sig,5))
    return der

cpdef void assign_za_D_derivative_cy(long[:] x, int i, double[:] za, double[:] zb, 
                             double[:] za_D_derivatives, double[:] zb_D_derivatives, double[:] Ea, double[:] Eb, double cab, double D):
    if i == l-1:
        za_D_derivatives[i] = 0
        return
    za_D_derivatives[i] = zb_D_derivatives[i-l]*cab*exp(-Ea[x[i]])
    
cpdef void assign_zb_D_derivative_cy(long[:] x, int i, double[:] za, double[:] zb, 
                             double[:] za_D_derivatives, double[:] zb_D_derivatives, double[:] Ea, double[:] Eb, double cab, double D):
    cdef double der = 0
    cdef int j
    if i == l-1:
        der += 0
    else:
        for j in range(0,i-l+1):
            der += za_D_derivatives[j]*cb_c(i-l-j, cab, D, l_p) + za[j]*cb_D_derivative_cy(i-l-j, D)
            der += zb_D_derivatives[j]*cab
    der *= exp(-Eb[x[i]])
    der += zb_D_derivatives[i-1]
    
    zb_D_derivatives[i] = der
    
    
cpdef void assign_za_cab_derivative_cy(long[:] x, int i, double[:] za, double[:] zb, 
                                    double[:] za_cab_derivatives, double[:] zb_cab_derivatives, double[:] Ea, double[:] Eb, double cab, double D):
    if i == l-1:
        za_cab_derivatives[i] = exp(-Ea[x[i]])
        return
    za_cab_derivatives[i] = exp(-Ea[x[i]])*(zb_cab_derivatives[i-l]*cab + zb[i-l])
    
cpdef void assign_zb_cab_derivative_cy(long[:] x, int i, double[:] za, double[:] zb, 
                                    double[:] za_cab_derivatives, double[:] zb_cab_derivatives, double[:] Ea, double[:] Eb, double cab, double D):
    cdef double der = 0
    cdef int j
    
    if i == l-1:
        der += 1
    else:
        for j in range(0,i-l+1):
            der += za_cab_derivatives[j]*cb_c(i-l-j, cab, D, l_p) + za[j]
            der += zb_cab_derivatives[j]*cab + zb[j]
    der *= exp(-Eb[x[i]])
    der += zb_cab_derivatives[i-1]
    
    zb_cab_derivatives[i] = der
    
    
def DP_Z_cy(double[:] args, long[:] x):
    
    cdef int L = len(x)

    cdef double[:] Ea = args[0:len(kmer_inx)]
    cdef double[:] Eb = args[len(kmer_inx):2*len(kmer_inx)]
    cdef double cab = args[-2]
    cdef D = args[-1]
    
    #initialization of statistical weigths
    cdef double[:] za = np.zeros(L)
    cdef double[:] zb = np.zeros(L)

    cdef int i
    for i in range(0,l-1):
        zb[i] = 1 

    #initialization of derivatives
    cdef double[:,::1] za_Ea_derivatives = np.zeros((len(kmer_inx),L))
    cdef double[:,::1] zb_Ea_derivatives = np.zeros((len(kmer_inx),L))

    cdef double[:,::1] za_Eb_derivatives = np.zeros((len(kmer_inx),L))
    cdef double[:,::1] zb_Eb_derivatives = np.zeros((len(kmer_inx),L))

    cdef double[:] za_D_derivatives = np.zeros(L)
    cdef double[:] zb_D_derivatives = np.zeros(L)

    cdef double[:] za_cab_derivatives = np.zeros(L)
    cdef double[:] zb_cab_derivatives = np.zeros(L)


    cdef int inx
    #dynamic programming calculation of z and derivatives 
    for i in range(l-1,L):
        #calculate statistical weights
        assign_za_cy(x, i, za, zb, Ea, Eb, cab, D)
        assign_zb_cy(x, i, za, zb, Ea, Eb, cab, D)
        
        #calculate derivatives
        for inx in range(len(kmer_inx)):
            assign_za_E_derivatives_cy(x, i, inx, za, zb, za_Ea_derivatives, zb_Ea_derivatives, za_Eb_derivatives, zb_Eb_derivatives, Ea, Eb, cab, D)
            #assign_zb_E_derivatives_cy(x, i, inx, za, zb, za_Ea_derivatives, zb_Ea_derivatives, za_Eb_derivatives, zb_Eb_derivatives, Ea, Eb, cab, D)
            assign_zb_E_derivatives_c(&x[0], i, inx, &za[0], &zb[0], L, l, l_p, 
                                      &za_Ea_derivatives[0,0], &zb_Ea_derivatives[0,0], &za_Eb_derivatives[0,0], &zb_Eb_derivatives[0,0], 
                                      &Ea[0], &Eb[0], cab, D)
        
        
        assign_za_D_derivative_cy(x, i, za, zb, za_D_derivatives, zb_D_derivatives, Ea, Eb, cab, D)
        assign_zb_D_derivative_cy(x, i, za, zb, za_D_derivatives, zb_D_derivatives, Ea, Eb, cab, D)
        
        assign_za_cab_derivative_cy(x, i, za, zb, za_cab_derivatives, zb_cab_derivatives, Ea, Eb, cab, D)
        assign_zb_cab_derivative_cy(x, i, za, zb, za_cab_derivatives, zb_cab_derivatives, Ea, Eb, cab, D)
        
    Z_x = zb[L-1] + np.sum(za)
    
    #derivative of Z(x)
    d_Ea = zb_Ea_derivatives[:,L-1] + np.sum(za_Ea_derivatives, axis=1)
    d_Eb = zb_Eb_derivatives[:,L-1] + np.sum(za_Eb_derivatives, axis=1)
    d_D = zb_D_derivatives[L-1] + np.sum(za_D_derivatives)
    d_cab = zb_cab_derivatives[L-1] + np.sum(za_cab_derivatives)
    
    
    gradient = np.concatenate([q.ravel() for q in [d_Ea, d_Eb, np.array([d_cab, d_D])]])
    
    return Z_x, gradient

### implementation of the LL object

In [51]:
class nLL:
    def __init__(self, seqs_p, seqs_bg):
        
        self.N_p = len(seqs_p)
        self.N_bg = len(seqs_bg)

        #calculate background probabilities:

        #include positive sequences in bg sequences if not there
        X_bg_t = list(set(seqs_p + seqs_bg))  #number of unique sequences
        
        counts = np.zeros(len(X_bg_t))
        for i, x in enumerate(X_bg_t):
            counts[i] = seqs_bg.count(x)
            
        counts = counts + 1 #pseudocount to make sure 
        counts = counts/np.sum(counts)

        p_bg = dict(zip(X_bg_t, counts))

        self.pbg_xp = np.array([p_bg[x] for x in seqs_p])
        self.pbg_xbg = np.array([p_bg[xbg] for xbg in seqs_bg])
        
        self.X_p = [seq2int_cy(x) for x in seqs_p]
        self.X_bg = [seq2int_cy(x) for x in seqs_bg]
        
        
    def __call__(self, args):
        
        #exp parameters to make sure they are positive
        args = np.exp(args)
        print(args[-10:])
    
        #implement LL and derivatives   
        z_x = np.zeros(self.N_p)
        d_z_x = np.zeros((2*len(kmer_inx)+2, self.N_p))

        z_xbg = np.zeros(self.N_bg)
        d_z_xbg = np.zeros((2*len(kmer_inx)+2, self.N_bg))


        for i, xp in enumerate(self.X_p):
            z_x[i], d_z_x[:,i] = DP_Z_cy(args, xp)
        print(z_x[:5])

        for i, xbg in enumerate(self.X_bg):
            z_xbg[i], d_z_xbg[:,i] = DP_Z_cy(args, xbg)
        print(z_xbg[:5])

        ll = np.sum(np.log(self.pbg_xp) + np.log(np.ones(self.N_p) - (np.ones(self.N_p)/z_x)))
        print("LL part 1: \t%f"%ll)
        ll -= self.N_p * logsumexp( np.log(self.pbg_xbg) + np.log(np.ones(self.N_bg) - (np.ones(self.N_bg)/z_xbg)) )
        print("LL part 1+2: \t%f"%ll)
        
        dll = np.sum(d_z_x/(z_x*(z_x - 1)), axis=1) 
        dll -= self.N_p * ( np.sum((self.pbg_xbg * d_z_xbg)/(z_xbg*z_xbg), axis=1 ) / np.sum(self.pbg_xbg*(np.ones(self.N_bg) - (np.ones(self.N_bg)/z_xbg))))
        #exp modify dLL
        dll = dll*args

        #regularize
        reg = 1e-5 
        ll -= np.sum(np.power(args[:-2],2)*reg)
        dll[:-2] -= 2*reg*args[:-2]

        print("final LL: \t%f"%ll)
        print(dll[-10:])
        print("============\n\n")
        return -ll, -dll 

In [8]:
Ea = np.zeros(len(kmer_inx)) + 1
Eb = np.zeros(len(kmer_inx)) + 1
D = cab = 1

parameters = np.concatenate([x.ravel() for x in [Ea, Eb, np.array([cab, D])]])

param2 = np.concatenate([x.ravel() for x in [Ea, Eb, np.array([cab, D])]])

In [104]:
p = 'TTGTGGTGTTAGTGTTAGTGTTAGGTTAGATCATTTGTCT'
DP_Z_cy(np.exp(param2), seq2int_cy(p))

(143948.06431303057,
 array([ 0.00000000e+000,  0.00000000e+000,  0.00000000e+000,
         0.00000000e+000,  0.00000000e+000,  0.00000000e+000,
         0.00000000e+000, -1.82605237e+003, -1.37119204e+003,
         0.00000000e+000,  0.00000000e+000,  0.00000000e+000,
         0.00000000e+000,  0.00000000e+000,  0.00000000e+000,
        -4.53888529e+003,  0.00000000e+000,  0.00000000e+000,
         0.00000000e+000,  0.00000000e+000,  0.00000000e+000,
         0.00000000e+000,  0.00000000e+000,  0.00000000e+000,
         0.00000000e+000,  0.00000000e+000,  0.00000000e+000,
         0.00000000e+000, -1.62310089e+003,  0.00000000e+000,
        -5.78115692e+003,  0.00000000e+000,  0.00000000e+000,
         0.00000000e+000,  0.00000000e+000, -4.61177053e+003,
         0.00000000e+000,  0.00000000e+000,  0.00000000e+000,
         0.00000000e+000, -1.14767476e+003,  0.00000000e+000,
         0.00000000e+000, -1.41543677e+003,  0.00000000e+000,
         0.00000000e+000, -4.93493105e+003, -6.72

In [9]:
p_array = []
def callb(x):
    print('next iteration')
    p_array.append(x)

In [62]:
nll_obj = nLL(plus,bg)

In [63]:
x_opt, fx, info = fmin_l_bfgs_b(nll_obj, x0=param2*2.5, callback=callb)

[12.18249396 12.18249396 12.18249396 12.18249396 12.18249396 12.18249396
 12.18249396 12.18249396 12.18249396 12.18249396]
[1.04656568 1.04656568 1.04656568 1.04656568 1.04656568]
[1.04656568 1.04656568 1.04656568 1.04656568 1.04656568]
LL part 1: 	-881.618792
LL part 1+2: 	-529.831737
final LL: 	-530.021705
[-2.13489781e+01  2.52718286e+00 -3.28581053e+01 -6.16222395e+00
  6.62192968e+00  1.12457056e+01 -3.65432367e+01 -1.06944165e+01
 -2.16404565e-14  0.00000000e+00]


[10.07240294 12.45989375  9.09080131 11.5317025  12.92283103 13.46628815
  8.79718483 11.07535218 12.18249396 12.18249396]
[3.32668195 2.87302748 1.54906427 2.43577512 2.03319641]
[1.09119774 1.14786235 1.20135401 1.14204152 1.93939353]
LL part 1: 	-649.919824
LL part 1+2: 	-475.615108
final LL: 	-475.807476
[  7.09102306  -0.48720916  15.2806013    2.53564522   0.6591706
   0.32789248   1.13990015   3.10324417 -15.08373773   0.        ]


next iteration
[10.50218038 12.41741589  9.88043657 11.69645577 12.91595246 13.4

In [64]:
info

{'grad': array([-1.23268918e-03, -1.70871335e-03, -6.77297588e-04, -3.33519753e-04,
        -2.42918473e-03, -1.51301812e-03, -2.02238732e-04, -1.24244817e-03,
        -3.50736038e-04, -3.30608195e-04, -1.53169528e-04, -1.42932174e-04,
        -7.02246601e-04, -3.94404881e-06, -6.90694791e-05, -1.85109077e-05,
        -1.88981204e-03, -1.36895001e-03, -4.59570208e-05, -1.18203514e-03,
         5.90833783e-04, -3.59782725e-04, -7.92324781e-04, -7.88655852e-04,
        -1.12334347e-04, -1.90351808e-04, -6.59264136e-04, -5.35578678e-04,
        -9.34060358e-05, -2.55732610e-03, -1.91422719e-04, -1.64190019e-04,
        -3.98642693e-04,  6.91506495e-06, -2.41705894e-04, -3.41042506e-04,
        -3.16404981e-04, -1.80001996e-03, -2.34289792e-04, -3.32024589e-04,
        -1.84468397e-03, -2.56946156e-04, -3.33365562e-04, -1.78395408e-04,
        -1.64076639e-04, -6.41406361e-04, -7.71053420e-05, -1.62098259e-04,
        -3.16196142e-04, -8.19137012e-04, -5.93235538e-05, -2.22805741e-04,
    

In [65]:
np.exp(x_opt)

array([1.54924578e+01, 1.42307138e+01, 1.61179183e+01, 1.68543261e+01,
       1.44375744e+01, 1.51239970e+01, 1.72933150e+01, 1.45435182e+01,
       1.66417898e+01, 1.68259349e+01, 1.77920146e+01, 1.78902851e+01,
       1.59565403e+01, 1.78162642e+01, 1.74421842e+01, 1.78693273e+01,
       1.39011382e+01, 1.54728454e+01, 1.79175367e+01, 1.51335868e+01,
       1.24806892e+01, 1.65986637e+01, 1.58306835e+01, 1.57384531e+01,
       1.77289683e+01, 1.74570691e+01, 1.62108274e+01, 1.61818583e+01,
       1.77067493e+01, 1.36317739e+01, 1.67832709e+01, 1.74247591e+01,
       1.65548928e+01, 1.78628078e+01, 1.77333542e+01, 1.60906532e+01,
       1.68942260e+01, 1.37458424e+01, 1.74835977e+01, 1.73981446e+01,
       1.27092811e+01, 1.76861054e+01, 1.65627375e+01, 1.74888380e+01,
       1.73851638e+01, 1.58967394e+01, 1.82040972e+01, 1.75292694e+01,
       1.71885917e+01, 1.57440478e+01, 1.76631744e+01, 1.80033348e+01,
       1.78093440e+01, 1.64599409e+01, 1.58832563e+01, 1.70604465e+01,
      

In [66]:
core1 = {}
for i in range(len(kmer_inx)):
    core1[inx_kmer[i]] = np.exp(x_opt)[i]

In [67]:
pd.Series(core1).sort_values(ascending=True)

ACC    12.480689
AGG    12.709281
CTC    13.631774
CCG    13.745842
AAC    13.901138
         ...    
GAC    17.917537
TAT    18.003335
GTG    18.204097
TGT    18.242167
AGT    18.488656
Length: 64, dtype: float64

In [68]:
core2 = {}
for i in range(len(kmer_inx)):
    core2[inx_kmer[i]] = np.exp(x_opt)[i+64]
pd.Series(core2).sort_values(ascending=True)

TAG      6.840137
TCA      8.151469
GTT      8.701158
CCG      8.725113
ACA      9.572784
          ...    
AGT     60.829646
GTC     64.195736
GTA     73.687411
TTA    105.892815
GGT    137.184645
Length: 64, dtype: float64